# Baseline on Street Level images

In [2]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from torchvision import datasets, transforms
import torchvision.models as models
import time
from tqdm import trange
from tqdm import tqdm
import torch.nn as nn
import copy
import torch.optim as optim
import pandas as pd
import os



In [3]:
def train_val_dataset(dataset, val_split=0.3):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

# define the path to the images
path_to_data = 'street_level_images/'

#defining a built-in tranform function
transform = transforms.Compose([transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])

# load the dataset
dataset = datasets.ImageFolder(path_to_data, transform=transform)
split_dataset = train_val_dataset(dataset)

In [52]:
# defining dataloader
batch_size = 128 # taken from baseline
dataloaders_dict = {x: torch.utils.data.DataLoader(split_dataset[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['train', 'val']}


In [24]:
# get folder names
locations = [name for name in os.listdir(path_to_data)]

# load features dataframe
features_df = pd.read_csv('dhs_final_labels.csv')

# make a target dict
target_dict = {}
test_keys = list(features_df['DHSID_EA'])
test_values = list(features_df['sanitation_index'])

target_dict = {test_keys[i]: test_values[i] for i in range(len(test_keys))}

<IPython.core.display.Javascript object>

In [45]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

def train_model(model, dataloaders, locations, target_dict,  criterion, optimizer,num_epochs=20, is_inception=False):
    since = time.time()

    val_acc_history = []
    train_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            with tqdm(dataloaders[phase],unit = 'batch') as tepoch:
              # Iterate over data.
              for inputs, labels in tepoch:
            
                  # get DHSID_EA id for alll labels
                  lb_ids = [locations[id] for id in labels]
                  # get feature values
                  labels = [target_dict[lb_id] for lb_id in lb_ids]

                  labels = torch.Tensor(labels)

                  #inputs = inputs.to(device)
                  #labels = labels.to(device)
                  print('labels',labels)

                  # zero the parameter gradients
                  optimizer.zero_grad()

                  # forward
                  # track history if only in train
                  with torch.set_grad_enabled(phase == 'train'):
                      # Get model outputs and calculate loss
                      # Special case for inception because in training it has an auxiliary output. In train
                      #   mode we calculate the loss by summing the final output and the auxiliary output
                      #   but in testing we only consider the final output.
                      
                    outputs = model(inputs)
                    print('outputs', outputs)
                    loss = criterion(outputs, labels)
                    print('loss', loss)

                    _, preds = torch.max(outputs, 1)
                    

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                      

                  # statistics
                  running_loss += loss.item() * inputs.size(0)
                  #getCategoricalAccuracy(preds, labels.data, class_dict)
                  running_corrects += torch.sum(preds == labels.data)

              epoch_loss = running_loss / len(dataloaders[phase].dataset)
              epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

              print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

              # deep copy the model
              if phase == 'val' and epoch_acc > best_acc:
                  best_acc = epoch_acc
                  best_model_wts = copy.deepcopy(model.state_dict())
              if phase == 'val':
                  val_acc_history.append(epoch_acc)
              if phase == 'train':
                  train_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, train_acc_history

cpu


In [54]:
# Setup the loss fxn
import time
import copy
criterion = nn.MSELoss()
epoch = 50 # from baseline
lr = 0.01 # from baseline



In [55]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

feature_extract = True

In [56]:

# ResNet 34
model_ft = models.resnet34(pretrained=True)
set_parameter_requires_grad(model_ft, True)
model_ft.fc = nn.Linear(512, 1)

input_size = 224
model_ft_balanced = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(params_to_update, lr=1e-4)

# Training a pretrained Resnet18 on a balanced dataset
model_ft, val_hist, hist  = train_model(model_ft, dataloaders_dict, locations, target_dict,   criterion, optimizer_ft,num_epochs= epoch)

C:\Users\nbelisse\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\nbelisse\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Params to learn:
	 fc.weight
	 fc.bias
Epoch 0/49
----------


  0%|          | 0/140 [00:00<?, ?batch/s]

labels tensor([3.0000, 3.4091, 5.0000, 3.3182, 5.0000, 3.0000, 4.9000, 3.2500, 5.0000,
        3.0909, 5.0000, 5.0000, 5.0000, 4.7273, 3.0000, 5.0000, 3.0909, 3.6818,
        3.7143, 5.0000, 3.0000, 3.0000, 3.2857, 5.0000, 5.0000, 4.1818, 3.9545,
        3.2857, 3.3182, 5.0000, 3.4091, 3.3182, 5.0000, 3.2727, 3.0000, 3.0000,
        5.0000, 3.0000, 5.0000, 3.0000, 5.0000, 3.0476, 3.0476, 5.0000, 5.0000,
        5.0000, 3.0000, 3.9091, 5.0000, 5.0000, 3.2857, 3.0833, 3.0000, 3.0000,
        5.0000, 3.2857, 3.0909, 5.0000, 3.7727, 3.7273, 5.0000, 3.4091, 3.0455,
        5.0000, 3.0000, 3.0000, 3.3810, 4.0000, 3.2609, 3.0455, 3.0000, 3.4500,
        5.0000, 3.0000, 5.0000, 4.3333, 3.0000, 4.2727, 5.0000, 5.0000, 3.8000,
        3.7143, 3.4286, 5.0000, 3.3810, 3.0000, 3.0000, 5.0000, 5.0000, 4.0909,
        5.0000, 3.0000, 3.2273, 5.0000, 3.0000, 3.2727, 3.0476, 5.0000, 3.2273,
        4.1364, 5.0000, 3.1429, 3.0000, 4.0909, 3.0000, 5.0000, 4.1429, 3.7273,
        3.3810, 5.0000, 3.4000, 3

C:\Users\nbelisse\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  1%|          | 1/140 [00:12<30:06, 13.00s/batch]

...

outputs tensor([[ 0.6932],
        [-0.0207],
        [ 0.4903],
        [ 0.5259],
        [-0.0563],
        [ 0.5875],
        [ 0.1028],
        [ 0.7487],
        [ 0.6066],
        [ 0.1397],
        [ 1.3457],
        [ 0.4008],
        [ 0.3368],
        [ 0.3565],
        [ 0.7378],
        [ 0.5287],
        [ 0.7695],
        [ 0.6379],
        [ 0.0708],
        [ 0.9839],
        [ 1.3036],
        [ 0.3823],
        [ 0.2575],
        [ 0.1084],
        [ 1.1936],
        [ 0.5102],
        [ 0.6090],
        [ 0.6964],
        [ 0.2651],
        [ 0.5814],
        [ 1.0980],
        [ 0.8345],
        [ 0.9034],
        [ 0.7193],
        [ 0.5856],
        [ 0.3862],
        [ 0.8161],
        [ 0.0104],
        [ 0.9198],
        [ 0.4422],
        [-0.3181],
        [ 1.0256],
        [ 0.5488],
        [ 0.9005],
        [ 0.8192],
        [-0.3550],
        [ 0.2327],
        [ 0.0900],
        [ 0.5271],
        [ 0.5507],
        [-0.1020],
        [ 0.4227],
    

  4%|▍         | 6/140 [00:45<15:28,  6.93s/batch]

outputs tensor([[ 0.4502],
        [ 0.7074],
        [ 0.4310],
        [ 0.7698],
        [ 0.5750],
        [ 1.2358],
        [ 0.0426],
        [ 0.3341],
        [ 0.4106],
        [ 0.8689],
        [ 0.3714],
        [ 0.7798],
        [ 0.6740],
        [ 1.0718],
        [ 0.9814],
        [ 0.5233],
        [-0.3094],
        [ 0.3271],
        [ 0.7271],
        [ 0.7488],
        [ 0.0856],
        [ 1.1403],
        [ 0.1634],
        [ 0.9833],
        [ 0.2816],
        [ 0.6400],
        [ 0.3258],
        [ 0.7880],
        [ 0.3811],
        [ 0.0560],
        [ 0.9568],
        [ 1.0310],
        [ 0.5419],
        [ 0.8092],
        [ 0.8629],
        [ 0.7968],
        [ 0.8707],
        [ 0.6073],
        [ 0.9473],
        [ 0.1669],
        [ 0.7625],
        [ 0.9289],
        [ 0.7841],
        [ 0.5892],
        [ 0.3863],
        [ 0.5057],
        [ 0.6784],
        [ 0.6393],
        [ 0.8371],
        [ 0.7444],
        [ 0.8761],
        [ 0.5886],
    

  4%|▍         | 6/140 [00:48<18:00,  8.06s/batch]


KeyboardInterrupt: 